In [1]:
import warnings

warnings.simplefilter("ignore")

import os
import sys

folder = os.getcwd().split("\\examples")[0]
sys.path.append(folder)

from sloyka import CommentsReply, VkPostGetter

INFO: Pandarallel will run on -1 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sandro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
owner_id = "-129354225" # группа "центральный район за комфортную среду обитания"
token = ''

In [3]:
posts = VkPostGetter.run(owner_id, token, 100, "2024-03-01")

2024-01-30 10:30:00
finished


In [6]:
posts.head(3)

,date,id,text,views.count,likes.count,reposts.count,link
id,,,,,,,
333157,2024-03-19 10:30:00,333157,Мы не об этом просили! На углу Восстания и Жук...,2157,44,6,https://vk.com/wall-129354225_315591?access_ke...
333061,2024-03-18 19:30:00,333061,Без вашей помощи мы не справимся! Открываем кр...,4563,80,28,https://vk.com/@mytndvor-programma2024|предвыб...
333033,2024-03-18 10:30:00,333033,Начинаем новую неделю с хороших новостей: в Пе...,4910,151,8,https://vk.com/wall-129354225_330987|нашей


In [4]:
# Вызов метода
post_ids = posts["id"].to_list()
comments = CommentsReply().run(owner_id=owner_id, post_ids=post_ids, access_token=token)

In [7]:
comments.head(3)

,id,date,text,post_id,parents_stack,likes_count
0,333159,2024-03-19 07:38:40,Может быть это переделывает исполнитель? За св...,333157.0,[],4.0
1,333161,2024-03-19 07:41:44,"[id6624566|Виталий], таких ""косяков"" не бывает.",333157.0,[333159],2.0
2,333189,2024-03-19 09:21:16,"[id8174482|Андрей], тут не соглашусь) у нас ка...",333157.0,[333159],1.0


In [5]:
posts.index = posts.id
comments_posts = comments.join(posts, on='post_id', rsuffix='_source')
comments_posts.head(3)

,id,date,text,post_id,parents_stack,likes_count,date_source,id_source,text_source,views.count,likes.count,reposts.count,link
0,333159,2024-03-19 07:38:40,Может быть это переделывает исполнитель? За св...,333157.0,[],4.0,2024-03-19 10:30:00,333157.0,Мы не об этом просили! На углу Восстания и Жук...,2157.0,44.0,6.0,https://vk.com/wall-129354225_315591?access_ke...
1,333161,2024-03-19 07:41:44,"[id6624566|Виталий], таких ""косяков"" не бывает.",333157.0,[333159],2.0,2024-03-19 10:30:00,333157.0,Мы не об этом просили! На углу Восстания и Жук...,2157.0,44.0,6.0,https://vk.com/wall-129354225_315591?access_ke...
2,333189,2024-03-19 09:21:16,"[id8174482|Андрей], тут не соглашусь) у нас ка...",333157.0,[333159],1.0,2024-03-19 10:30:00,333157.0,Мы не об этом просили! На углу Восстания и Жук...,2157.0,44.0,6.0,https://vk.com/wall-129354225_315591?access_ke...
